In [ ]:
%pylab inline
import os
from matplotlib import pyplot as plt
import numpy as np
import logbook

In [ ]:
exp_name = "23_plot_path"
exp = logbook.Experiment(exp_name)
log = exp.get_logger()
path_dirs = [
    "archive/20_find_path_length/2017-05-05 13:28:24 dbe068d"
]
stats_file = "path_stats.csv"

In [ ]:
def get_mean_path_length(filename):
    all_nodes = set()
    total = 0
    count = 0
    with open(filename, "rb") as f:
        for row in f:
            project_id, source, target, length = row.rstrip().split(",")
            if source == target:
                continue
            total += int(length)
            count += 1
            all_nodes.add(int(target))
            all_nodes.add(int(source))
    node_count = len(all_nodes)
    mean_length = float(total) / float(count)
    return mean_length

In [ ]:
def get_harmonic_path_length(filename):
    all_nodes = set()
    total = 0.0
    with open(filename, "rb") as f:
        for row in f:
            project_id, source, target, length = row.rstrip().split(",")
            if source == target:
                continue
            total += 1.0 / float(length)
            all_nodes.add(int(target))
            all_nodes.add(int(source))
    node_count = len(all_nodes)
    pair_count = node_count * (node_count - 1)
    mean_length = float(pair_count) / total
    return mean_length

In [ ]:
[]all_means = []
all_harmonic = 
with open(exp.get_filename(stats_file), "wb") as out:
    out.write("project_id,path_mean,path_harmonic\n")
    for path_dir in path_dirs:
        for i, path_file in enumerate(os.listdir(path_dir)):
            if path_file.endswith('path_length.csv'):
                project_id = path_file.split("-")[0]
                log.info("Loading path file: %s" % path_file)
                path_mean = get_mean_path_length(os.path.join(path_dir, path_file))
                path_harmonic = get_harmonic_path_length(os.path.join(path_dir, path_file))
                all_means.append(path_mean)
                all_harmonic.append(path_harmonic)
                out.write("%s,%s,%s\n" % (project_id, repr(path_mean), repr(path_harmonic)))
                out.flush()

In [ ]:
plt.figure(figsize=(8,10))
plt.subplot(2,1,1)
plt.hist(all_means, 60)
plt.xlabel("Mean path length")
plt.ylabel("Frequency")
plt.title("editor_count < 5551, excluding infinite length")
plt.subplot(2,1,2)
plt.hist(all_harmonic, 60)
plt.xlabel("Harmonic mean path length")
plt.ylabel("Frequency")
plt.title("editor_count < 5551")
plt.tight_layout()